# COGS 108 - Data Checkpoint

# Names

- Tatiana Bimatova
- Huy Le
- Margarita Leonova
- Brandon Luong
- Noel Molina

<a id='research_question'></a>
# Research Question

*What characteristics of beer (aroma, taste, price, etc.) influence the popularity of beer brands? Certain characteristics such as aroma and taste will be dependent on the consumer’s personal preference.*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: Beer Data Set
- Link to the dataset: https://www.kaggle.com/datasets/stephenpolozoff/top-beer-information?select=beer_data_set.csv
- Number of observations: 5558

This dataset contains the reviews of different beers. The data contained includes information on the beer, such as the average rating, bitterness, and a description, as well as a tasting profile for each beer.

- Dataset Name: Beer Descriptors Simplified
- Link to the dataset: https://www.kaggle.com/datasets/stephenpolozoff/top-beer-information?select=beer_data_set.csv
- Number of observations: 209

This dataset contains the definitions of each beer feature. These definitions represent the impact for each feature.

THe Beer Data Set will be our main source of data, while the Beer Descriptors Simplified data will be used to standardize our data in the Beer Data Set.

# Setup

In [50]:
import pandas as pd
import numpy as np

In [51]:
beer_data_set = pd.read_csv('beer_data_set.csv')
descriptors = pd.ExcelFile('Beer Descriptors Simplified.xlsx')
beer_descriptors = []#pd.read_excel('Beer Descriptors Simplified.xlsx')
for sheet in descriptors.sheet_names:
    df = descriptors.parse(sheet)
    beer_descriptors.append(df)
description_df = pd.concat(beer_descriptors, ignore_index=False, axis=1)

# Data Cleaning


Our data cleaning process:

We first looked at the columns and dropped unnecessary columns and renamed columns for simplicity. We then checked for missing values in the data set and dropped the rows that contained missing beer entries (there were not many missing values). There are 11 descriptors for each beer. To handle the many descriptors, we grouped each into one of three categores: 'flavor and aroma', 'mouthfeel', 'taste'. Many beer styles were the same but contained little difference, for example: 'IPA - Hazy' and 'IPA - Summer'. We grouped these similiar beer styles so tha we can better group the beers themselves, for example: grouping those IPAs into one overall IPA style. Finally, the descriptors for each bear had a score based on the number of descriptor words used for that beer. Since there were a different number of potential descriptor words for each, we standardized each score to be able to better compare the beers. 

In [52]:
# remove impact columns from the descriptions dataframe
description_df = description_df.drop(description_df.filter(regex='Impact').columns, axis=1)
# lowercase the column names
description_df.columns = description_df.columns.str.lower()
# rename columns to match the descriptors in both data sets
description_df = description_df.rename(columns = {'fruity':'fruits', 'astringent':'astringency', 'alcoholic':'alcohol'})
description_df

,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,acetic,acacia,absinthe,ahtanum,acacia,acetic,caviar,berries,ahtanum,absinthe,amaranth
1,acid,acasia,agave,amarillo,acasia,acid,cracker,berry,amarillo,allspice,azrael
2,acidic,amaranth,alcohol,anjilanaka,apple,acidic,crackers,blabaer,apollo,anise,barley
3,acidified,amargor,alcoholic,apollo,apples,acidified,crackery,blackberries,azacca,aniseed,barleymalt
4,acidifying,big,alcoholized,ashtray,apricot,acidifying,salt,blackberry,bitter,annatto,barleys
...,...,...,...,...,...,...,...,...,...,...,...
204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,oreos
205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,seedy
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,toffe
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tofee


In [53]:
# lowercase the column names
beer_data_set.columns = beer_data_set.columns.str.lower()
# drop 'style key' column and rename 'name' column to 'beer'
beer_data_set = (beer_data_set.drop(columns = ['style key'])
        .rename(columns = {'name': 'beer'})
)
beer_data_set

,beer,key,style,brewery,description,abv,ave rating,min ibu,max ibu,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,Amber,251,Altbier,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,13,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,14,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,13,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,21,69,10,63,120,14,0,19,36,15,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5553,Red Nose Winter Ale,696,Winter Warmer,Natty Greene's Pub & Brewing Co.,Notes:,6.8,3.59,35,50,8,44,24,19,52,21,0,26,21,96,77
5554,Our Special Ale 2019 (Anchor Christmas Ale),697,Winter Warmer,Anchor Brewing Company,Notes:This is the forty-fifth annual Our Speci...,6.9,3.79,35,50,22,73,9,58,87,23,0,48,91,44,104
5555,Fish Tale Winterfish,698,Winter Warmer,Fish Brewing Company / Fishbowl Brewpub,Notes:,7.5,3.76,35,50,11,36,50,70,72,59,0,81,110,18,73
5556,"He'Brew Hanukkah, Chanukah: Pass The Beer",699,Winter Warmer,Shmaltz Brewing Company,Notes:Chanukah Beer pours a rich crystal clear...,8.0,3.61,35,50,6,64,30,57,78,15,1,28,57,23,129


In [54]:
# checking for missing values
beer_data_set.isna().sum()

beer           2
key            0
style          0
brewery        0
description    0
abv            0
ave rating     0
min ibu        0
max ibu        0
astringency    0
body           0
alcohol        0
bitter         0
sweet          0
sour           0
salty          0
fruits         0
hoppy          0
spices         0
malty          0
dtype: int64

In [55]:
# drop rows that contain missing beer entries
beer_data_set = beer_data_set[beer_data_set['beer'].notna()]
beer_data_set

,beer,key,style,brewery,description,abv,ave rating,min ibu,max ibu,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,Amber,251,Altbier,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,13,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,14,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,13,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,21,69,10,63,120,14,0,19,36,15,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5553,Red Nose Winter Ale,696,Winter Warmer,Natty Greene's Pub & Brewing Co.,Notes:,6.8,3.59,35,50,8,44,24,19,52,21,0,26,21,96,77
5554,Our Special Ale 2019 (Anchor Christmas Ale),697,Winter Warmer,Anchor Brewing Company,Notes:This is the forty-fifth annual Our Speci...,6.9,3.79,35,50,22,73,9,58,87,23,0,48,91,44,104
5555,Fish Tale Winterfish,698,Winter Warmer,Fish Brewing Company / Fishbowl Brewpub,Notes:,7.5,3.76,35,50,11,36,50,70,72,59,0,81,110,18,73
5556,"He'Brew Hanukkah, Chanukah: Pass The Beer",699,Winter Warmer,Shmaltz Brewing Company,Notes:Chanukah Beer pours a rich crystal clear...,8.0,3.61,35,50,6,64,30,57,78,15,1,28,57,23,129


In [56]:
# group the descriptors into three groups: 'flavor and aroma', 'mouthfeel', 'taste'
mouthfeel = ['astringency',	'body',	'alcohol']
taste = ['bitter', 'sweet', 'sour', 'salty']
flavor_and_aroma = ['fruits', 'hoppy', 'spices', 'malty']
beer_data_set['mouthfeel'] = beer_data_set[mouthfeel].sum(axis=1)
beer_data_set['taste'] = beer_data_set[taste].sum(axis=1)
beer_data_set['flavor_and_aroma'] = beer_data_set[flavor_and_aroma].sum(axis=1)
beer_data_set

/var/folders/nb/d_1vl9354vs7vm_nj7q4n7gh0000gn/T/ipykernel_4158/523050184.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beer_data_set['mouthfeel'] = beer_data_set[mouthfeel].sum(axis=1)
/var/folders/nb/d_1vl9354vs7vm_nj7q4n7gh0000gn/T/ipykernel_4158/523050184.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beer_data_set['taste'] = beer_data_set[taste].sum(axis=1)
/var/folders/nb/d_1vl9354vs7vm_nj7q4n7gh0000gn/T/ipykernel_4158/523050184.py:7: SettingWithCopyWarning: 
A value is trying to be set on 

,beer,key,style,brewery,description,abv,ave rating,min ibu,max ibu,astringency,...,sweet,sour,salty,fruits,hoppy,spices,malty,mouthfeel,taste,flavor_and_aroma
0,Amber,251,Altbier,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,13,...,74,33,0,33,57,8,111,54,154,209
1,Double Bag,252,Altbier,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,12,...,55,16,0,24,35,12,84,87,104,155
2,Long Trail Ale,253,Altbier,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,14,...,43,11,0,10,54,4,62,57,96,130
3,Doppelsticke,254,Altbier,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,13,...,101,18,1,49,40,16,119,99,167,224
4,Scurry,255,Altbier,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,21,...,120,14,0,19,36,15,218,100,197,288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5553,Red Nose Winter Ale,696,Winter Warmer,Natty Greene's Pub & Brewing Co.,Notes:,6.8,3.59,35,50,8,...,52,21,0,26,21,96,77,76,92,220
5554,Our Special Ale 2019 (Anchor Christmas Ale),697,Winter Warmer,Anchor Brewing Company,Notes:This is the forty-fifth annual Our Speci...,6.9,3.79,35,50,22,...,87,23,0,48,91,44,104,104,168,287
5555,Fish Tale Winterfish,698,Winter Warmer,Fish Brewing Company / Fishbowl Brewpub,Notes:,7.5,3.76,35,50,11,...,72,59,0,81,110,18,73,97,201,282
5556,"He'Brew Hanukkah, Chanukah: Pass The Beer",699,Winter Warmer,Shmaltz Brewing Company,Notes:Chanukah Beer pours a rich crystal clear...,8.0,3.61,35,50,6,...,78,15,1,28,57,23,129,100,151,237


In [62]:
# aggregated the style of beers into common groups. ex: IPA, Lager, etc.
beer_data_set['style'] = beer_data_set['style'].apply(lambda x: x.split('-')[0])
beer_data_set

/var/folders/nb/d_1vl9354vs7vm_nj7q4n7gh0000gn/T/ipykernel_4158/1126729567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beer_data_set['style'] = beer_data_set['style'].apply(lambda x: x.split('-')[0])


,beer,key,style,brewery,description,abv,ave rating,min ibu,max ibu,astringency,...,sweet,sour,salty,fruits,hoppy,spices,malty,mouthfeel,taste,flavor_and_aroma
0,Amber,251,Altbier,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,13,...,74,33,0,33,57,8,111,54,154,209
1,Double Bag,252,Altbier,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,12,...,55,16,0,24,35,12,84,87,104,155
2,Long Trail Ale,253,Altbier,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,14,...,43,11,0,10,54,4,62,57,96,130
3,Doppelsticke,254,Altbier,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,13,...,101,18,1,49,40,16,119,99,167,224
4,Scurry,255,Altbier,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,21,...,120,14,0,19,36,15,218,100,197,288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5553,Red Nose Winter Ale,696,Winter Warmer,Natty Greene's Pub & Brewing Co.,Notes:,6.8,3.59,35,50,8,...,52,21,0,26,21,96,77,76,92,220
5554,Our Special Ale 2019 (Anchor Christmas Ale),697,Winter Warmer,Anchor Brewing Company,Notes:This is the forty-fifth annual Our Speci...,6.9,3.79,35,50,22,...,87,23,0,48,91,44,104,104,168,287
5555,Fish Tale Winterfish,698,Winter Warmer,Fish Brewing Company / Fishbowl Brewpub,Notes:,7.5,3.76,35,50,11,...,72,59,0,81,110,18,73,97,201,282
5556,"He'Brew Hanukkah, Chanukah: Pass The Beer",699,Winter Warmer,Shmaltz Brewing Company,Notes:Chanukah Beer pours a rich crystal clear...,8.0,3.61,35,50,6,...,78,15,1,28,57,23,129,100,151,237


In [66]:
# number of beers for each style
beer_data_set['style'].value_counts()

Lager                              899
Stout                              400
Wheat Beer                         350
IPA                                350
Porter                             300
Bock                               250
Strong Ale                         200
Sour                               196
Lambic                             163
Pale Ale                           150
Red Ale                            150
Farmhouse Ale                      150
Brown Ale                          150
Pilsner                            150
Blonde Ale                         100
Barleywine                         100
Bitter                             100
Mild Ale                           100
Scottish Ale                        50
Smoked Beer                         50
Scotch Ale / Wee Heavy              50
Rye Beer                            50
Rye Beer                            50
Tripel                              50
Quadrupel (Quad)                    50
Pumpkin Beer             

In [47]:
#check for duplicate beers
beer_data_set['beer'].value_counts()

Oktoberfest                                                                    15
Porter                                                                         12
Smoked Porter                                                                  11
Roggenbier                                                                     11
Christmas Ale                                                                   8
                                                                               ..
Olimpia Kvas                                                                    1
Family Secret Customized, Fragrant (Boiled, Flavored) Household Kvass (Семе     1
Drevlyansky Kvass                                                               1
Family Secret Borodino Household Kvass                                          1
Frosted Frog Christmas Ale - Barrel-Aged                                        1
Name: beer, Length: 5317, dtype: int64

In [48]:
# aggregate the data so that there is only one review for each beer
beer_data_set = beer_data_set.groupby('beer').mean().reset_index()
beer_data_set

,beer,key,abv,ave rating,min ibu,max ibu,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty,mouthfeel,taste,flavor_and_aroma
0,(Organic) Dunkel,883.0,4.8,3.88,15.0,25.0,34.0,51.0,11.0,38.0,68.0,10.0,0.0,24.0,43.0,12.0,115.0,96.0,116.0,194.0
1,17 Grande Réserve,4570.0,10.0,4.26,25.0,50.0,6.0,66.0,28.0,14.0,127.0,40.0,0.0,78.0,11.0,50.0,83.0,100.0,181.0,222.0
2,Barriquée (Italy Version),5441.0,8.0,4.21,10.0,25.0,30.0,41.0,19.0,11.0,74.0,148.0,2.0,88.0,19.0,19.0,39.0,90.0,235.0,165.0
3,Barriquée (Sour Version),5426.0,8.0,4.01,10.0,25.0,43.0,40.0,10.0,6.0,86.0,160.0,1.0,98.0,30.0,18.0,37.0,93.0,253.0,183.0
4,Blanca,5037.0,4.6,3.36,10.0,15.0,0.0,2.0,0.0,2.0,1.0,1.0,0.0,3.0,1.0,0.0,7.0,2.0,4.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312,Über Alt,292.0,8.5,3.96,25.0,50.0,32.0,64.0,28.0,47.0,135.0,49.0,2.0,65.0,55.0,15.0,182.0,124.0,233.0,317.0
5313,überPils,2872.0,7.6,3.66,30.0,65.0,25.0,48.0,31.0,35.0,61.0,30.0,6.0,44.0,59.0,15.0,74.0,104.0,132.0,192.0
5314,Šamp - Pivní Sekt,1070.0,5.0,4.04,10.0,35.0,10.0,12.0,3.0,10.0,14.0,22.0,0.0,24.0,16.0,3.0,15.0,25.0,46.0,58.0
5315,Žaibo,2475.0,9.5,2.72,15.0,40.0,7.0,28.0,49.0,13.0,92.0,40.0,1.0,40.0,15.0,12.0,44.0,84.0,146.0,111.0


In [25]:
# check if the values in the descriptors aren't greater than the amount of descriptive words
check_desc = beer_data_set.set_index('beer').iloc[:, 5:].reset_index()
for desc in description_df.columns:
    check_desc[desc] = description_df[desc].dropna().count() - check_desc[desc]

# we can see that there are some negative values in the descriptor fields. This implies that the same word was used more than once to describe the beer.
check_desc

,beer,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty,mouthfeel,taste,flavor_and_aroma
0,(Organic) Dunkel,55.0,46.0,53.0,48.0,60.0,101.0,13.0,150.0,105.0,87.0,94.0,96.0,116.0,194.0
1,17 Grande Réserve,83.0,31.0,36.0,72.0,1.0,71.0,13.0,96.0,137.0,49.0,126.0,100.0,181.0,222.0
2,Barriquée (Italy Version),59.0,56.0,45.0,75.0,54.0,-37.0,11.0,86.0,129.0,80.0,170.0,90.0,235.0,165.0
3,Barriquée (Sour Version),46.0,57.0,54.0,80.0,42.0,-49.0,12.0,76.0,118.0,81.0,172.0,93.0,253.0,183.0
4,Blanca,89.0,95.0,64.0,84.0,127.0,110.0,13.0,171.0,147.0,99.0,202.0,2.0,4.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312,Über Alt,57.0,33.0,36.0,39.0,-7.0,62.0,11.0,109.0,93.0,84.0,27.0,124.0,233.0,317.0
5313,überPils,64.0,49.0,33.0,51.0,67.0,81.0,7.0,130.0,89.0,84.0,135.0,104.0,132.0,192.0
5314,Šamp - Pivní Sekt,79.0,85.0,61.0,76.0,114.0,89.0,13.0,150.0,132.0,96.0,194.0,25.0,46.0,58.0
5315,Žaibo,82.0,69.0,15.0,73.0,36.0,71.0,12.0,134.0,133.0,87.0,165.0,84.0,146.0,111.0


In [26]:
# the number of impact words for each beer feature
num_desc_words = np.invert(description_df.isna()).sum()


mouthfeel = ['astringency',	'body',	'alcohol']
taste = ['bitter', 'sweet', 'sour', 'salty']
flavor_and_aroma = ['fruits', 'hoppy', 'spices', 'malty']
# the number of descriptors for each group
num_mouthfeel = np.invert(description_df[mouthfeel].isna()).sum().sum()
num_taste = np.invert(description_df[taste].isna()).sum().sum()
num_flavor = np.invert(description_df[flavor_and_aroma].isna()).sum().sum()

In [27]:
# standardize the descriptors and descriptor groups
beer_data_set['astringency'] = beer_data_set['astringency'].apply(lambda x: x  / num_desc_words.loc['astringency'])

beer_data_set['body'] = beer_data_set['body'].apply(lambda x: x  / num_desc_words.loc['body'])

beer_data_set['alcohol'] = beer_data_set['alcohol'].apply(lambda x: x  / num_desc_words.loc['alcohol'])

beer_data_set['bitter'] = beer_data_set['bitter'].apply(lambda x: x  / num_desc_words.loc['bitter'])

beer_data_set['sweet'] = beer_data_set['sweet'].apply(lambda x: x  / num_desc_words.loc['sweet'])

beer_data_set['sour'] = beer_data_set['sour'].apply(lambda x: x  / num_desc_words.loc['sour'])

beer_data_set['salty'] = beer_data_set['salty'].apply(lambda x: x  / num_desc_words.loc['salty'])

beer_data_set['fruits'] = beer_data_set['fruits'].apply(lambda x: x  / num_desc_words.loc['fruits'])

beer_data_set['hoppy'] = beer_data_set['hoppy'].apply(lambda x: x  / num_desc_words.loc['hoppy'])

beer_data_set['spices'] = beer_data_set['spices'].apply(lambda x: x  / num_desc_words.loc['spices'])

beer_data_set['malty'] = beer_data_set['malty'].apply(lambda x: x  / num_desc_words.loc['malty'])

beer_data_set['mouthfeel'] = beer_data_set['mouthfeel'].apply(lambda x: x  / num_mouthfeel)

beer_data_set['taste'] = beer_data_set['taste'].apply(lambda x: x  / num_taste)

beer_data_set['flavor_and_aroma'] = beer_data_set['flavor_and_aroma'].apply(lambda x: x  / num_taste)

In [28]:
# final clean data set
final = beer_data_set
final

,beer,key,abv,ave rating,min ibu,max ibu,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty,mouthfeel,taste,flavor_and_aroma
0,(Organic) Dunkel,883.0,4.8,3.88,15.0,25.0,0.382022,0.525773,0.171875,0.441860,0.531250,0.090090,0.000000,0.137931,0.290541,0.121212,0.550239,0.384,0.343195,0.573964
1,17 Grande Réserve,4570.0,10.0,4.26,25.0,50.0,0.067416,0.680412,0.437500,0.162791,0.992188,0.360360,0.000000,0.448276,0.074324,0.505051,0.397129,0.400,0.535503,0.656805
2,Barriquée (Italy Version),5441.0,8.0,4.21,10.0,25.0,0.337079,0.422680,0.296875,0.127907,0.578125,1.333333,0.153846,0.505747,0.128378,0.191919,0.186603,0.360,0.695266,0.488166
3,Barriquée (Sour Version),5426.0,8.0,4.01,10.0,25.0,0.483146,0.412371,0.156250,0.069767,0.671875,1.441441,0.076923,0.563218,0.202703,0.181818,0.177033,0.372,0.748521,0.541420
4,Blanca,5037.0,4.6,3.36,10.0,15.0,0.000000,0.020619,0.000000,0.023256,0.007812,0.009009,0.000000,0.017241,0.006757,0.000000,0.033493,0.008,0.011834,0.032544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312,Über Alt,292.0,8.5,3.96,25.0,50.0,0.359551,0.659794,0.437500,0.546512,1.054688,0.441441,0.153846,0.373563,0.371622,0.151515,0.870813,0.496,0.689349,0.937870
5313,überPils,2872.0,7.6,3.66,30.0,65.0,0.280899,0.494845,0.484375,0.406977,0.476562,0.270270,0.461538,0.252874,0.398649,0.151515,0.354067,0.416,0.390533,0.568047
5314,Šamp - Pivní Sekt,1070.0,5.0,4.04,10.0,35.0,0.112360,0.123711,0.046875,0.116279,0.109375,0.198198,0.000000,0.137931,0.108108,0.030303,0.071770,0.100,0.136095,0.171598
5315,Žaibo,2475.0,9.5,2.72,15.0,40.0,0.078652,0.288660,0.765625,0.151163,0.718750,0.360360,0.076923,0.229885,0.101351,0.121212,0.210526,0.336,0.431953,0.328402
